<a href="https://colab.research.google.com/github/amodsgit/AmodTheCoder/blob/main/Workout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('/content/drive/MyDrive/fitness_survey.csv')

# Print the first few rows of the data
print(data.head(2))


                         Timestamp  Age Your Gender Your Height   Your Weight  \
0  2023/06/23 10:00:49 PM GMT+5:30   23        Male          NaN           70   
1  2023/06/24 10:09:36 PM GMT+5:30   24      Female          5.4           60   

  What is your job type (Desk Job / Student / Athlete)  \
0                                            Student     
1                                            Student     

   How many days a week do you like to exercise? (0-7)  \
0                                                  2     
1                                                  3     

   What is your preferred duration in minutes  \
0                                          60   
1                                          10   

  What are your fitness goals (Weight loss / Weight gain / Building muscle mass / Improving cardiovascular health / Increasing stamina/ endurance / Improving mental health)  \
0                                        Weight gain                              

In [ ]:
# Create "Injury Status" column
data['Injury Status'] = data['Do you have any existing injuries?'].map({'No': 0, 'Yes': 1})

# Fill NaNs in the "If yes, specify the injuries from the provided list" column
data['If yes, specify the injuries from the provided list (list: Lower Back/ Pain/Injury / Shoulder Injury / Knee Injury / Wrist Injury / Ankle Injury / Hip Injury)'] = data['If yes, specify the injuries from the provided list (list: Lower Back/ Pain/Injury / Shoulder Injury / Knee Injury / Wrist Injury / Ankle Injury / Hip Injury)'].fillna('No injury')


data = data.drop(columns=['Do you have any existing injuries?'])


In [ ]:
# Drop unnecessary columns
data = data.drop(columns=['Timestamp', 'Your Height ', 'Do you have any other health conditions that we should know about? If yes, mention those. Eg: asthma, Wheezing'])


In [ ]:
print(data.head(2))

   Age Your Gender  Your Weight  \
0   23        Male           70   
1   24      Female           60   

  What is your job type (Desk Job / Student / Athlete)  \
0                                            Student     
1                                            Student     

   How many days a week do you like to exercise? (0-7)  \
0                                                  2     
1                                                  3     

   What is your preferred duration in minutes  \
0                                          60   
1                                          10   

  What are your fitness goals (Weight loss / Weight gain / Building muscle mass / Improving cardiovascular health / Increasing stamina/ endurance / Improving mental health)  \
0                                        Weight gain                                                                                                                           
1                            Improving menta

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# Fill NaNs in the 'Recommended Exercises ()' column with 'None'
data['Recommended Exercises ()'] = data['Recommended Exercises ()'].fillna('None')

# Now continue with splitting and binarization
data['Recommended Exercises ()'] = data['Recommended Exercises ()'].str.split(',')

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the 'Recommended Exercises ()' column
exercises = mlb.fit_transform(data.pop('Recommended Exercises ()'))

# Convert the transformed exercises data into a DataFrame and add it to the original data
data = data.join(pd.DataFrame(exercises, columns=mlb.classes_))



In [ ]:
data = pd.get_dummies(data, columns=['Your Gender', 'What is your job type (Desk Job / Student / Athlete)', 'Do you prefer Indoor or Outdoor workouts? ', 'What type of workout do you prefer? (Body weight workouts (Eg: Push-ups, Sit-ups, Planks, Squats etc.) / Gym workouts (Eg: Bench press, Deadlifts, Treadmill exercises, Leg press etc./ No preference ', 'If yes, specify the injuries from the provided list (list: Lower Back/ Pain/Injury / Shoulder Injury / Knee Injury / Wrist Injury / Ankle Injury / Hip Injury)'])


In [ ]:

data = pd.get_dummies(data, columns=["What are your fitness goals (Weight loss / Weight gain / Building muscle mass / Improving cardiovascular health / Increasing stamina/ endurance / Improving mental health)"])


In [ ]:
print(data.head(1))

   Age  Your Weight  How many days a week do you like to exercise? (0-7)  \
0   23           70                                                  2     

   What is your preferred duration in minutes  Injury Status  B1  B10  B11  \
0                                          60              0   1    0    0   

   B12  B13  ...  \
0    0    0  ...   

   If yes, specify the injuries from the provided list (list: Lower Back/ Pain/Injury / Shoulder Injury / Knee Injury / Wrist Injury / Ankle Injury / Hip Injury)_Shoulder Injury  \
0                                                  0                                                                                                                                

   If yes, specify the injuries from the provided list (list: Lower Back/ Pain/Injury / Shoulder Injury / Knee Injury / Wrist Injury / Ankle Injury / Hip Injury)_Shoulder Injury;Knee Injury  \
0                                                  0                                         

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# Define your features and labels
X = data.drop(mlb.classes_, axis=1)  # dropping the columns representing exercises
Y = data[mlb.classes_]

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Random Forest
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(X_train, Y_train)
Y_pred_rf = multi_target_forest.predict(X_test)
acc_rf = accuracy_score(Y_test, Y_pred_rf)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

micro_precision_scores = [precision_score(Y_test, Y_pred_rf, average='micro')
                      ]

micro_recall_scores = [recall_score(Y_test, Y_pred_rf, average='micro')]

micro_f1_scores = [f1_score(Y_test, Y_pred_rf, average='micro')]


In [ ]:
jaccard_scores = [jaccard_score(Y_test, Y_pred_rf, average='samples')]


In [ ]:
rf_index = 0

print(f"Metrics for Random Forest:")
print(f"Micro-Averaged Precision: {micro_precision_scores[rf_index]}")
print(f"Micro-Averaged Recall: {micro_recall_scores[rf_index]}")
print(f"Micro-Averaged F1 Score: {micro_f1_scores[rf_index]}")
print(f"Jaccard Similarity Coefficient: {jaccard_scores[rf_index]}")


Metrics for Random Forest:
Micro-Averaged Precision: 0.9458333333333333
Micro-Averaged Recall: 0.8165467625899281
Micro-Averaged F1 Score: 0.8764478764478765
Jaccard Similarity Coefficient: 0.6707317073170732


In [ ]:
from joblib import dump

# save the model to disk
filename = 'random_forest_model.joblib'
dump(multi_target_forest, filename)


['random_forest_model.joblib']